Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape( (-1, image_size, image_size, num_channels)).astype(np.float32) # -1 implies the dimension is inferred
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32) # Labels need to be 1-hot float encodings
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [8]:
batch_size = 16
patch_size = 5  # 5x5 patches, each of depth 1
depth = 16      # 16 of them
num_hidden = 64

# 28x28x1 grayscale image input, same padding, stride of 2
# => 16 14 x 14 outputs from the first layer

# Second layer uses another 16 5x5 patches, same padding, stride of 2
# => 16 7 x 7 outputs from the second layer

# Third layer uses 64 length-7*7*16 patches (i.e. it is a fully-connected layer)

# Fourth layer uses 10 length-64 patches (i.e. it is a fully-connected layer)

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data): # Ah, so you CAN set up the model within the graph
        conv    = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME') # First layer convolution
        hidden  = tf.nn.relu(conv + layer1_biases) # ... Passed through ReLUs
        conv    = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME') # Second layer convolution
        hidden  = tf.nn.relu(conv + layer2_biases) # ... Passed through ReLUs
        shape   = hidden.get_shape().as_list()      # Flatten output to pass to fully-connected layer
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden  = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases # Return 10 values - the logits
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss   = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # ^ Pass the logits through the softmax function for class probabilities
    # Compute the cross-entropy to determine the distance between the class probabilities and ground truth
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data. (i.e. to feed back during training)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset)) # Wow, it is so much easier this way!
    test_prediction  = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
        print('Minibatch loss at step %d: %f' % (step, l))
        print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
        print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 1000: 0.761046
Minibatch accuracy: 75.0%
Validation accuracy: 83.5%
Test accuracy: 89.7%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [13]:
train_dataset.shape

(200000, 28, 28, 1)

In [28]:
image_size   = train_dataset.shape[1:3]
num_channels = 1
batch_size   = 32
num_classes  = 10
patch_size   = 5
n_patches_1  = 16
n_patches_2  = 16

In [ ]:
# Input => Batch size x 28x28x1 images
# Same padding, stride of 2, max pooling of size 2, 16 patches
# 14x14 before pooling -> 7x7 after pooling
# Batch_size x 7 x 7 x 16 in the next layer

# 7 x 7 x 16 layer with same padding, stride of 1, no pooling, 16 patches
# => Output is also 7 x 7 x 16 

# We'll have one fully-connected layer to begin with

# Set up a graph
my_graph = tf.Graph()

with my_graph.as_default():
    # Placeholders for data
    tf_training_images = tf.placeholder(tf.float32, [batch_size, image_size[0], image_size[1], num_channels])
    tf_training_labels = tf.placeholder(tf.float32, [batch_size, num_classes])
    tf_valid_images    = tf.constant(valid_dataset)
    tf_test_images     = tf.constant(test_images)
    
    # A network's like onions... They both have LAYERS
    weights1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, n_patches_1]))
    biases1  = tf.Variable(tf.zeros([n_patches_1]))
    
    weights2 = tf.Variable(tf.truncated_normal([patch_size // 4, patch_size // 4, n_patches_1, n_patches_2]))
    biases2  = tf.Variable(tf.zeros([n_patches_2]))
    
    weights3 = tf.Variable(tf.truncated_normal([patch_size // 4 * patch_size // 4 * n_patches_2, n_classes]))
    biases3  = tf.Variable(tf.truncated_normal([n_classes]))
    
    def model(data):
        conv1   = tf.conv2d(images, weights1, [1, 1, 1, 1], padding='SAME')
        pool1   = tf.nn.max_pool(conv1, ksize = [2, 2], strides = [1, 2, 2, 1])
        hidden1 = tf.relu(pool1 + biases1) # Notice we have 1 bias per patch - the pooling doesn't affect the number of biases
        
        conv2   = tf.conv2d(hidden1, weights2, [1, 1, 1, 1], padding='SAME')
        hidden2 = tf.relu(conv2 + biases2)
        
        hidden2 = tf.reshape(hidden, [batch_size, patch_size // 4 * patch_size // 4 * n_patches_2])
        activation = tf.matmul(hidden2, weights3) + biases3
        return tf.nn.relu(activation)
    
    # Optimizer and loss and feedback info

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---